In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot  as plt

In [2]:
from sklearn.cluster import KMeans, DBSCAN
from sklearn.metrics import silhouette_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import NearestNeighbors

In [3]:
import folium
import re
from geopy.distance import great_circle
from shapely.geometry import MultiPoint
import geopandas as gpd

In [4]:
df=pd.read_csv('/Users/mackbookpro/US_CarAccident/US_Accidents_Dec21_updated.csv') #put relevant file path

FileNotFoundError: [Errno 2] No such file or directory: '.../US_Accidents_Dec21_updated.csv'

In [ ]:
df.drop(df.columns.difference(['Start_Lat','Start_Lng', 'Start_Time', 'ID']), 1, inplace=True)

In [ ]:
df['Year'] = pd.to_datetime(df['Start_Time']).dt.year

In [ ]:
coords=df.loc[:,['Start_Lat','Start_Lng']].values
df['coords'] = [x for x in coords]
df.head()

,ID,Start_Time,Start_Lat,Start_Lng,Year,coords
0,A-1,2016-02-08 00:37:08,40.108910,-83.092860,2016,"[40.10891, -83.09286]"
1,A-2,2016-02-08 05:56:20,39.865420,-84.062800,2016,"[39.86542, -84.0628]"
2,A-3,2016-02-08 06:15:39,39.102660,-84.524680,2016,"[39.10266, -84.52468]"
3,A-4,2016-02-08 06:51:45,41.062130,-81.537840,2016,"[41.06213, -81.53784]"
4,A-5,2016-02-08 07:53:43,39.172393,-84.492792,2016,"[39.172393, -84.49279200000002]"


In [ ]:
kms_per_radian = 6371.0088
epsilon = 0.05 / kms_per_radian 
db = DBSCAN(eps=epsilon, min_samples=60, algorithm='ball_tree', metric='haversine').fit(np.radians(coords))
cluster_labels = db.labels_
num_clusters = len(set(cluster_labels))
clusters = pd.Series([coords[cluster_labels == n] for n in range(num_clusters)])

In [ ]:
print('Number of clusters: {}'.format(num_clusters))
print(f'Number of outliers found: {len(cluster_labels[cluster_labels==-1])}')

Number of clusters: 4017
Number of outliers found: 2423012


In [ ]:
plt.scatter(coords[:, 0],coords[:,1], c = cluster_labels, cmap= "plasma") 
plt.xlabel("Latitude") 
plt.ylabel("Longitude") 
plt.show() 

In [ ]:
clusters_centroids = [MultiPoint(x).centroid for x in clusters]
crs = {'init': 'epsg:4326'}
cgdf = gpd.GeoDataFrame(clusters, crs=crs, geometry=clusters_centroids)
good_cdgf = cgdf[ ~cgdf['geometry'].is_empty ]

good_cdgf.plot()

In [ ]:
"""def create_map(df,cluster_labels):

    plt_map=folium.Map(location=[df.Start_Lat.mean(),df.Start_Lng.mean()],zoom_start=10,
                 tiles='OpenStreetMap')
    for _,row in df.iterrows():
        folium.CircleMarker(
            location=[row.Start_Lat,row.Start_Lng],
            radius=5,
            popup=re.sub(r'[^a-zA-Z ]+','',row.ID),
            color='blue',
            fill=True
        ).add_to(plt_map)
        
    return plt_map"""

"def create_map(df,cluster_labels):\n\n    plt_map=folium.Map(location=[df.Start_Lat.mean(),df.Start_Lng.mean()],zoom_start=10,\n                 tiles='OpenStreetMap')\n    for _,row in df.iterrows():\n        folium.CircleMarker(\n            location=[row.Start_Lat,row.Start_Lng],\n            radius=5,\n            popup=re.sub(r'[^a-zA-Z ]+','',row.ID),\n            color='blue',\n            fill=True\n        ).add_to(plt_map)\n        \n    return plt_map"

In [ ]:
"""plt_map=create_map(df, cluster_labels)
plt_map"""

'plt_map=create_map(df, cluster_labels)\nplt_map'

In [ ]:
df['Cluster'] = cluster_labels
df.shape
#df.head()

(2845342, 7)

In [ ]:
df.drop(df[df['Cluster'] == -1].index, inplace = True)
df.head()

,ID,Start_Time,Start_Lat,Start_Lng,Year,coords,Cluster
115,A-116,2016-02-11 14:53:15,39.17736,-84.48730,2016,"[39.17736, -84.4873]",0
257,A-258,2016-02-17 10:54:38,39.17736,-84.48730,2016,"[39.17736, -84.4873]",0
593,A-594,2016-03-02 16:10:42,39.05654,-84.54283,2016,"[39.05654000000001, -84.54283000000002]",1
606,A-607,2016-03-02 20:16:10,39.17736,-84.48730,2016,"[39.17736, -84.4873]",0
722,A-723,2016-03-08 08:55:43,39.05654,-84.54283,2016,"[39.05654000000001, -84.54283000000002]",1


In [ ]:
df.shape

(422330, 7)

In [ ]:
out_df = df
out_df = out_df.groupby('Cluster').agg(lambda x: x.tolist())
out_df.head()

,ID,Start_Time,Start_Lat,Start_Lng,Year,coords
Cluster,,,,,,
0,"[A-116, A-258, A-607, A-805, A-38595, A-54237,...","[2016-02-11 14:53:15, 2016-02-17 10:54:38, 201...","[39.17736, 39.17736, 39.17736, 39.17736, 39.17...","[-84.4873, -84.4873, -84.4873, -84.4873, -84.4...","[2016, 2016, 2016, 2016, 2016, 2016, 2016, 201...","[[39.17736, -84.4873], [39.17736, -84.4873], [..."
1,"[A-594, A-723, A-757, A-762, A-967, A-36743, A...","[2016-03-02 16:10:42, 2016-03-08 08:55:43, 201...","[39.05654000000001, 39.05654000000001, 39.0565...","[-84.54283000000002, -84.54283000000002, -84.5...","[2016, 2016, 2016, 2016, 2016, 2016, 2016, 201...","[[39.05654000000001, -84.54283000000002], [39...."
2,"[A-991, A-992, A-1056, A-1560, A-1917, A-2170,...","[2016-03-22 20:07:32, 2016-03-22 21:40:18, 201...","[37.881943, 37.881038, 37.88124000000001, 37.8...","[-122.307987, -122.307788, -122.30814, -122.30...","[2016, 2016, 2016, 2016, 2016, 2016, 2016, 201...","[[37.881943, -122.307987], [37.881038, -122.30..."
3,"[A-998, A-1175, A-1968, A-1973, A-2359, A-2400...","[2016-03-23 06:45:09, 2016-03-28 06:43:13, 201...","[37.31648, 37.31648, 37.31648, 37.31648, 37.31...","[-121.96746, -121.96746, -121.96746, -121.9674...","[2016, 2016, 2016, 2016, 2016, 2016, 2016, 201...","[[37.31648, -121.96746], [37.31648, -121.96746..."
4,"[A-1003, A-2981, A-108673, A-109986, A-124920,...","[2016-03-23 08:52:43, 2016-05-04 14:58:22, 201...","[37.377536, 37.37737, 37.37737, 37.37737, 37.3...","[-121.945376, -121.94545, -121.94545, -121.945...","[2016, 2016, 2016, 2017, 2016, 2016, 2016, 201...","[[37.377536, -121.945376], [37.37737, -121.945..."


In [ ]:
out_df['Count'] = out_df.Year.apply(len) #number of accidents per specific cluster
out_df.head()

,ID,Start_Time,Start_Lat,Start_Lng,Year,coords,Count
Cluster,,,,,,,
0,"[A-116, A-258, A-607, A-805, A-38595, A-54237,...","[2016-02-11 14:53:15, 2016-02-17 10:54:38, 201...","[39.17736, 39.17736, 39.17736, 39.17736, 39.17...","[-84.4873, -84.4873, -84.4873, -84.4873, -84.4...","[2016, 2016, 2016, 2016, 2016, 2016, 2016, 201...","[[39.17736, -84.4873], [39.17736, -84.4873], [...",66
1,"[A-594, A-723, A-757, A-762, A-967, A-36743, A...","[2016-03-02 16:10:42, 2016-03-08 08:55:43, 201...","[39.05654000000001, 39.05654000000001, 39.0565...","[-84.54283000000002, -84.54283000000002, -84.5...","[2016, 2016, 2016, 2016, 2016, 2016, 2016, 201...","[[39.05654000000001, -84.54283000000002], [39....",63
2,"[A-991, A-992, A-1056, A-1560, A-1917, A-2170,...","[2016-03-22 20:07:32, 2016-03-22 21:40:18, 201...","[37.881943, 37.881038, 37.88124000000001, 37.8...","[-122.307987, -122.307788, -122.30814, -122.30...","[2016, 2016, 2016, 2016, 2016, 2016, 2016, 201...","[[37.881943, -122.307987], [37.881038, -122.30...",275
3,"[A-998, A-1175, A-1968, A-1973, A-2359, A-2400...","[2016-03-23 06:45:09, 2016-03-28 06:43:13, 201...","[37.31648, 37.31648, 37.31648, 37.31648, 37.31...","[-121.96746, -121.96746, -121.96746, -121.9674...","[2016, 2016, 2016, 2016, 2016, 2016, 2016, 201...","[[37.31648, -121.96746], [37.31648, -121.96746...",77
4,"[A-1003, A-2981, A-108673, A-109986, A-124920,...","[2016-03-23 08:52:43, 2016-05-04 14:58:22, 201...","[37.377536, 37.37737, 37.37737, 37.37737, 37.3...","[-121.945376, -121.94545, -121.94545, -121.945...","[2016, 2016, 2016, 2017, 2016, 2016, 2016, 201...","[[37.377536, -121.945376], [37.37737, -121.945...",61


In [ ]:
out_df.shape

(4016, 7)

In [ ]:
out_csv = out_df.to_csv('.../output.csv')